## Notebook with code used to transfer existing data from MongoDB acquired during writing thesis

In [3]:
from pymongo import ASCENDING, GEOSPHERE, MongoClient
import pandas as pd
from alive_progress import alive_bar
from shapely.geometry import Point, mapping
import shapely
import json
from os import listdir
from os.path import isfile, join
from tqdm import tqdm
from tqdm.contrib.concurrent import process_map
import geopandas as gpd

In [4]:
from db_connector.models import Area, TrainLabel, Vector

In [5]:
client = MongoClient('mongodb://localhost:27017/')

In [6]:
db = client.osmDataDB

In [7]:
coll_cities = db.cities

## Migrating cities into sqlite database

In [8]:
cities = [a for a in coll_cities.find({ 'accepted': True })]

In [9]:
cities[0]

{'_id': ObjectId('608c8e85d9325f7b4aa354ee'),
 'city_id': 1,
 'city': 'Antwerpen',
 'osm_areas_ids': [59518],
 'geometry': {'type': 'Polygon',
  'coordinates': [[[4.2175769, 51.3738851],
    [4.2182598, 51.3722094],
    [4.2199279, 51.3683223],
    [4.220477, 51.3673435],
    [4.2220663, 51.3645103],
    [4.2267461, 51.3597586],
    [4.2325255, 51.3575718],
    [4.242049, 51.3539671],
    [4.2594513, 51.3463655],
    [4.2630991, 51.3445695],
    [4.2658028, 51.3431487],
    [4.2672153, 51.3421246],
    [4.2682919, 51.3412454],
    [4.2697939, 51.3393687],
    [4.2712101, 51.3370899],
    [4.2726263, 51.3338993],
    [4.273313, 51.3317542],
    [4.2735705, 51.3303867],
    [4.2737851, 51.3281342],
    [4.2736992, 51.3249966],
    [4.273313, 51.3199814],
    [4.2730555, 51.3152607],
    [4.2733559, 51.3129002],
    [4.2739138, 51.3108882],
    [4.2754588, 51.3081251],
    [4.2789411, 51.3031754],
    [4.2808661, 51.3012835],
    [4.2833981, 51.2999955],
    [4.2853288, 51.299298],
    [4

In [10]:
for city in cities:
    shp = shapely.geometry.shape(city['geometry'])
    area = Area()
    area.name = city['city']
    area.lon = shp.centroid.x
    area.lat = shp.centroid.y
    area.shape = city['geometry']
    area.save()

## Seeding database with vectors

In [11]:
from h3 import h3
import math

In [12]:
coll_relations = db.relations
coll_stations = db.stationsInCities

In [13]:
RESOLUTION = 11

edge_size = h3.edge_length(RESOLUTION, unit='m')
h = edge_size * math.sqrt(3) / 2
k_neighbours = math.ceil(2000 / (h*2))
print(edge_size, h, k_neighbours)
NEIGHBORS = k_neighbours

24.910561 21.57317864852189 47


In [14]:
def getHexes(city_id):
    station_indexes = set()
    all_indexes = set()
    for st in tqdm([s for s in coll_stations.find({ 'city_id': city_id })], desc=f"{city['city']} - Loading station hexes"):
        pt = shapely.geometry.shape(st['geometry'])
        h = h3.geo_to_h3(pt.y, pt.x, RESOLUTION)
        indexes = h3.hex_range(h, NEIGHBORS)
        station_indexes.add(h)
        all_indexes.update(indexes)
    return station_indexes, all_indexes

In [15]:
def check_hex(h):
    global shp
    polygons = h3.h3_set_to_multi_polygon([h], geo_json=True)
    outlines = [loop for polygon in polygons for loop in polygon]
    polyline = [outline + [outline[0]] for outline in outlines][0]
    polygon = shapely.geometry.Polygon(polyline)
    if shp.intersects(polygon):
        return {'hex_id': h, 'geometry': polygon}
    return None

In [16]:
hexes_dict = {}
for city in coll_cities.find({ 'accepted': True }):
    shp = shapely.geometry.shape(city['geometry'])
    buffered_polygon = shp.buffer(0.005)

    hexes_dict[city['city']] = {}
    stations, non_stations = getHexes(city['city_id'])
    hexes_shp = process_map(check_hex, non_stations, max_workers=8, desc=f"{city['city']} - Filtering hexes", chunksize = 1)
    hexes_dict[city['city']]['stations'] = stations
    hexes_dict[city['city']]['non_stations'] = set([h['hex_id'] for h in hexes_shp if h is not None])
    print(city['city'], len(hexes_dict[city['city']]['stations']), len(hexes_dict[city['city']]['non_stations']))

Barcelona - Loading station hexes:   5%|▍         | 25/502 [00:00<00:01, 248.05it/s]

Antwerpen 292 56967


Berlin - Loading station hexes:   0%|          | 0/305 [00:00<?, ?it/s]

Barcelona 497 42436


Berlin - Filtering hexes: 100%|██████████| 122989/122989 [00:26<00:00, 4615.27it/s]


...

Wrocław 215 126558


Zurich - Loading station hexes:  10%|█         | 17/170 [00:00<00:00, 165.84it/s]

Zaragoza 129 32227


Zurich - Filtering hexes: 100%|██████████| 70203/70203 [00:20<00:00, 3431.49it/s]


Zurich 170 70203


In [17]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

In [18]:
class BaseCountCategoryEmbedding():
    def __init__(self):
        client = MongoClient('mongodb://localhost:27017/')
        db = client.osmDataDB
        self.coll_cities = db.cities
        self.coll_relations_filtered = db.relationsFiltered

    def fit(self):
        categories = self.coll_relations_filtered.find({}, {'_id':0, 'category': 1}).distinct('category')
        self.vectorizer = CountVectorizer()
        self.vectorizer.fit(categories)
       

    def _transform(self, hex):
        polygons = h3.h3_set_to_multi_polygon([hex], geo_json=True)
        # flatten polygons into loops.
        outlines = [loop for polygon in polygons for loop in polygon]
        polyline = [outline + [outline[0]] for outline in outlines][0]
        polygon = shapely.geometry.Polygon(polyline)

        c = MongoClient('mongodb://localhost:27017/')

        relations = c.osmDataDB.relationsFiltered.find({
            "geometry": {
                "$geoIntersects": {
                    "$geometry": shapely.geometry.mapping(polygon)
                }
            }
        },
        {
            '_id': 0,
            'category': 1
        })
        categories = [r['category'] for r in relations]
        if len(categories) > 0:
            functions = ' '.join(categories)
            vector = self.vectorizer.transform([functions]).toarray().reshape((-1,))
        else:
            # print(f'No relations for hex {hex["hex_id"]}')
            vector = np.zeros(shape=(len(self.vectorizer.get_feature_names()),))
        return np.array(vector).reshape((-1,))

    def transform(self, hex):
        vector = self._transform(hex)
        return np.array(vector).reshape((-1,))

In [19]:
embedder = BaseCountCategoryEmbedding()
embedder.fit()

In [20]:
from db_connector.db_definition import db as sqlite_db

In [21]:
def parse_hex(h):
    global embedder
    return embedder.transform(h)

In [22]:
for city, hexes in hexes_dict.items():
    city_obj = Area.get(Area.name == city)
    h3_indexes = [h for h in hexes['non_stations']]
    stations = [h for h in hexes['stations']]
    vectors = process_map(parse_hex, h3_indexes, max_workers=8, desc=f"{city} - Vectorizing hexes", chunksize = 1)
    with sqlite_db.atomic():
        for hex, vector in tqdm(zip(h3_indexes, vectors), desc=f"{city} - Saving vectors", total=len(h3_indexes)):
            tv_obj = {
                'area': city_obj,
                'hex_id': hex
            }
            label_obj = {
                'area': city_obj,
                'has_station': hex in stations,
                'hex_id': hex
            }
            for i, v in enumerate(vector):
                tv_obj[f'v{i}'] = int(v)
            Vector.insert(**tv_obj).execute()
            TrainLabel.insert(**label_obj).execute()

Zurich - Saving vectors: 100%|██████████| 70203/70203 [00:38<00:00, 1842.78it/s]
